# Create BRISE dataset for POTATO

In [2]:
from potato.dataset.dataset import Dataset
from potato.models.trainer import GraphTrainer

In [25]:
import os
import json

GOLD_ATTRIBUTES = "gold_attributes"
GOLD = "labels_gold"
ANNOTATED_ATTRIBUTES = "annotated_attributes"

def load_data(dir_path, only_gold=True):
    data = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "rt") as f:
            doc = json.load(f)
            if only_gold and not doc[GOLD]:
                continue
            for sen in doc["sens"].values():
                if doc[GOLD]:
                    labels = list(set(sen[GOLD_ATTRIBUTES].keys()))
                else:
                    labels = list(set(sen[ANNOTATED_ATTRIBUTES].keys()))
                data.append({
                    "sen_id": sen["id"],
                    "text": sen["text"],
                    "labels": labels,
                })
    return data

In [26]:
NOT = "NOT"

def get_sentences(data, attribute):
    sentences = []
    for sen in data:
        if attribute in sen["labels"]:
            sentences.append((sen["text"], attribute))
        else:
            sentences.append((sen["text"], NOT))       
    return sentences

In [63]:
import pickle

def get_graphs(dataset, graph_path):
    if not os.path.exists(graph_path):
        dataset.set_graphs(dataset.parse_graphs(graph_format="ud"))
    else:
        dataset.load_graphs(graph_path)
    df = dataset.to_dataframe()
    if not os.path.exists(graph_path):
        with open(graph_path, "wb") as f:
            pickle.dump(df.graph, f)
    return df

## Load data - only gold

In [28]:
TRAIN_DIR = "../../../../data/train"
VAL_DIR = "../../../../data/valid"

train_data = load_data(TRAIN_DIR, only_gold=True)
val_data = load_data(VAL_DIR, only_gold=True)

In [29]:
len(train_data)

1154

In [32]:
len(val_data)

100

In [33]:
train_data[10:15]

[{'sen_id': '8025_7_0',
  'text': 'Bestimmungen ohne Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_8_0',
  'text': 'Für das gesamte Plangebiet wird bestimmt: Dächer dürfen die festgesetzte Gebäudehöhe um höchstens 4,5 m überragen.',
  'labels': ['GebaeudeHoeheArt',
   'PlangebietAllgemein',
   'AbschlussDachMaxBezugGebaeude']},
 {'sen_id': '8025_9_0',
  'text': 'Im Bauland/Wohngebiet darf die Bruttogeschoßfläche aller Geschoße, die ganz oder teilweise über dem anschließenden Gelände liegen, insgesamt höchstens 11.500 m² betragen.',
  'labels': ['WidmungUndZweckbestimmung', 'Flaechen']},
 {'sen_id': '8025_10_0',
  'text': 'Bestimmungen mit Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'labels': ['Planzeichen']},
 {'sen_id': '8025_11_0',
  'text': 'Für die mit BB1 bezeichneten Grundflächen wird bestimmt: Die Unterbrechung der geschlossenen Bauweise ist zulässig',
  'labels': ['Planzeichen', 'UnterbrechungGeschlos

In [36]:
val_data[15:20]

[{'sen_id': '7080k_2_0',
  'text': 'Für die rechtliche Bedeutung der roten Planzeichen ist die beiliegende „Zeichenerklärung für den Flächenwidmungsplan und den Bebauungsplan“ (§§ 4 und 5 BO für Wien) vom 1. Oktober 2001 maßgebend, die einen Bestandteil dieses Beschlusses bildet.',
  'labels': []},
 {'sen_id': '7080k_3_0',
  'text': 'Die Bestimmungen 3.1.1 und 3.1.2.',
  'labels': []},
 {'sen_id': '7080k_3_1',
  'text': 'des Plandokumentes 7080 haben für das gesamte Plangebiet keine Gültigkeit.',
  'labels': []},
 {'sen_id': '7080k_4_0',
  'text': 'Gemäß § 5 (4) der Bauordnung für Wien wird für Teile des Plangebietes mit eigener Kennzeichnung im Plan (BB) bestimmt:',
  'labels': []},
 {'sen_id': '7080k_5_0',
  'text': 'Auf den mit BB17 bezeichneten Flächen wird die bauliche Ausnutzbarkeit mit maximal 45 % beschränkt.',
  'labels': ['Planzeichen', 'Flaechen']}]

## Planzeichen - only gold - ud

In [74]:
t_sentences = get_sentences(train_data, "Planzeichen")

In [75]:
t_sentences[10:15]

[('Bestimmungen ohne Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'Planzeichen'),
 ('Für das gesamte Plangebiet wird bestimmt: Dächer dürfen die festgesetzte Gebäudehöhe um höchstens 4,5 m überragen.',
  'NOT'),
 ('Im Bauland/Wohngebiet darf die Bruttogeschoßfläche aller Geschoße, die ganz oder teilweise über dem anschließenden Gelände liegen, insgesamt höchstens 11.500 m² betragen.',
  'NOT'),
 ('Bestimmungen mit Bezeichnung des Geltungsbereichs mit dem Planzeichen BB:',
  'Planzeichen'),
 ('Für die mit BB1 bezeichneten Grundflächen wird bestimmt: Die Unterbrechung der geschlossenen Bauweise ist zulässig',
  'Planzeichen')]

In [77]:
t_dataset = Dataset(t_sentences, label_vocab={"NOT":0, "Planzeichen": 1})

In [78]:
GRAPH_PATH = "planzeichen_gold_train_ud.pickle"

df_train = get_graphs(t_dataset, GRAPH_PATH)

In [68]:
df_train.head()

,text,label,label_id,graph
0,MAGISTRAT DER STADT WIEN MA 21 Stadtteilplanun...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14,..."
1,Der Gemeinderat hat in seiner Sitzung am 29. A...,NOT,0,"(1, 3, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,"Bezirk, Kat. Großjedlersdorf I werden unter An...",NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Die bisherigen Flächenwidmungspläne und Bebauu...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 13, 10, 11, 12,..."
4,Gemäß § 4 und § 5 der BO für Wien sowie § 48 d...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [79]:
TRAIN_DATASET = "train_planzeichen_gold_dataset"

df_train.to_pickle(TRAIN_DATASET)

## Val dataset

In [80]:
v_sentences = get_sentences(val_data, "Planzeichen")

In [81]:
v_sentences[10:15]

[('MAGISTRAT DER STADT WIEN MA 21 B Stadtteilplanung und Flächennutzung Süd-Nordost Plandokument 7080K Abänderung des Flächenwidmungsplanes und des Bebauungsplanes',
  'NOT'),
 ('Der Gemeinderat hat in seiner Sitzung am 01. Dezember 2005, Pr. Zl. 4075/2005–GSV, den folgenden Beschluss gefasst: In Abänderung des Flächenwidmungsplanes und des Bebauungsplanes für das im Antragsplan Nr. 7080K mit der rot strichpunktierten Linie oder den in roter Schrift als „Plangebietsgrenze“ bezeichneten Fluchtlinien umschriebene Gebiet zwischen Hödlgasse, Ziedlergasse, Linienzug 1-2 und Scherbangasse im 23.',
  'NOT'),
 ('Bezirk, Kat. G. Atzgersdorf werden unter Anwendung des § 1 der BO für Wien folgende Bestimmungen getroffen:',
  'NOT'),
 ('Die roten Planzeichen gelten als neu festgesetzt;', 'NOT'),
 ('die schwarzen Planzeichen behalten ihre Rechtskraft, sofern sie nicht rot überdeckt, durchkreuzt oder durchstrichen sind.',
  'NOT')]

In [82]:
v_dataset = Dataset(v_sentences, label_vocab={"NOT":0, "Planzeichen": 1})

In [84]:
GRAPH_PATH = "planzeichen_gold_valid_ud.pickle"

df_val = get_graphs(v_dataset, GRAPH_PATH)

In [85]:
df_val.head()

,text,label,label_id,graph
0,MAGISTRAT DER STADT WIEN MA 21 A Stadtteilplan...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11, 12, 13,..."
1,Der Gemeinderat hat in seiner Sitzung am 23. N...,NOT,0,"(1, 3, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 13, 12,..."
2,"Bezirk, Kat. G. Leopoldstadt werden unter Anwe...",NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,Die roten Planzeichen gelten als neu festgesetzt.,NOT,0,"(1, 0, 2, 3, 4, 7, 5, 6, 8)"
4,die schwarzen Planzeichen behalten ihre Rechts...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13,..."


In [86]:
VALID_DATASET = "valid_planzeichen_gold_dataset"

df_val.to_pickle(VALID_DATASET)